Available at http://www.comp.nus.edu.sg/~cs3244/1910/11.colab

![Machine Learning](https://www.comp.nus.edu.sg/~cs3244/1910/img/banner-1910.png)
---
See **Credits** below for acknowledgements and rights.  For NUS class credit, you'll need to do the corresponding _Assessment_ in [CS3244 in Coursemology](http://coursemology.org/courses/1677) by the respective deadline (as in Coursemology). 

**You must acknowledge that your submitted Assessment is your independent work, see questions in the Assessment at the end.**

**Learning Outcomes for this Week** 

After watching the videos and completing the exercises for this week, you should be able to:

* Have a basic understanding of unsupervised learning;
* Understand different unsupervised learning algorithms for:
  * Dimensionality reduction: For aiding visualization, exploratory analysis;  
  * Clustering: Finding latent patterns.
* Implement and understand $k$ means clustering;
* Understand the Expectation Maximization meta-algorithm;
  * and how estimating the Gaussian Mixture Model exemplifies it;
* Apply unsupervised learning on real-world data.



*Welcome to the Week $11$ Python notebook.* This week we will learn about **Unsupervised Learning**. We introduce the topic in the lecture videos, and will be reviewing this material in the ninth tutorial.

In the notebook, we go through popular algorithms in unsupervised learning; namly *PCA*, *$k$-means*, *Gaussian Mixture Model*. In the pre-tutorial notebook, we look into an example of PCA and implement $k$-means from scratch. We look through modification of $k$-means algorithm in the post-tutorial portion. Also, we look into a few problems of using $k$-means and try to overcome them using Gaussian Mixture Model.

---
# Week 11: Pre-Tutorial Work

Watch the videos for Week $11$ Pre and answer the following question.

**Your Turn (Question 1):** Which of the following is not an example of unsupervised learning?

_Choose from: Social network analysis to define group of friends, Organizing computing clusters based on similar event patterns and processes, Answering multiple choice questions_

## 1 Programming : Face Recognition using PCA

In the pre-videos, we learned about **PCA** and **Eigenfaces**, so now we will implement a **face recognition** algorithm using these methods with `scikit-learn`. For this example, we will use the image dataset [Labeled Faces in the Wild](http://vis-www.cs.umass.edu/lfw/lfw.pdf) (LFW), which contains images with labels. We will use a percentage of these images to train a **Logistic Regression** classifier and try to predict unseen images. Before doing that we will use *PCA* to reduce the dimensionality of the images, and then use those for our model. 

Ready?  Let's get started on our implementation!

### .a Setup
The usual formalities.  Let's get our `import`s done to get our toolkit online.

In [0]:
# import necessary libraries
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

### .b Load Dataset
We will now download our `lfw_people` dataset using `sklearn`. Note that, in the dataset, we have different number of images for each person, some may have $70$ images, some $10$ and so on. We are loading only those people's images who have at least **$60$** images in the dataset.

In [0]:
# Download the data and load it in as a set of numpy arrays
# min_faces_per_person limits the dataset to have the image with at least that amount
lfw_people = fetch_lfw_people(min_faces_per_person=60, resize=0.4)

# introspect the images arrays to find the shapes (for plotting)
n_samples, h, w = lfw_people.images.shape

X = lfw_people.data
n_features = X.shape[1]

# the label to predict is the ID of the person
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

print ("\nTotal dataset size:")
print ("n_samples: %d" % n_samples)
print ("n_features: %d" % n_features)
print ("n_classes: %d" % n_classes)

Now that we have loaded the dataset, let's ensure that we know what you're working with (Rule 1: know thy dataset).  Let's examine several faces! Put an integer from $1$ to `n_samples` in the `visualize_face` variable and see whose image it is!

In [0]:
######### Put in an integer from 1 to n_samples ###########
visualize_face = 40

# Plotting the grey image of the above index
plt.figure(figsize=(3.6, 4.8))
plt.imshow(X[visualize_face].reshape((h, w)), cmap=plt.cm.gray)
plt.xticks(())
plt.yticks(())

We will split our dataset into training and testing using `sklearn`.

In [0]:
# split into a training (75%) and testing (25%) set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

### .c Applying PCA
The next step is dimensionality reduction. From the above output we have seen that there are in total $1850$ features in the dataset, which is a lot (!). We will use scikit-learn's  `PCA` class to perform the dimensionality reduction. For that, we have to select the **number of components**, i.e. the output dimensionality (number of *eigenvectors* to project onto). We can tune this parameter to get a better result. For starters, let's use $150$ components.

In [0]:
######### You may change this parameter later ###########
# Number of components for PCA
n_components = 150
########################################################

print("Extracting the top %d eigenfaces from %d faces" % (n_components, X_train.shape[0]))

pca = PCA(n_components=n_components, svd_solver='randomized', whiten=True, random_state=42).fit(X_train)

# reconstructing the images from pca output --> eigenfaces
eigenfaces = pca.components_.reshape((n_components, h, w))

# Applying pca to training and test set
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

### .d Training Linear Classifier on the PCA results
Now that we have reduced the number of dimensions, let's use the transformed data to train our SVM classifier.  

We'll employ a Logistic Regression with Limited-memory BFGS (`lbfgs`) as optimizer and multinomial loss. You can read more about the different parameters of Logistic Regression from [`sklearn` documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).


OK. Let's train our classifier!


In [0]:
print("Fitting the classifier to the training set")

# creating Logistic Regression classifier instance
clf = LogisticRegression(random_state=11, solver='lbfgs', multi_class='multinomial', max_iter=1000)

# Training our classfier
clf = clf.fit(X_train_pca, y_train)

# Quantitative evaluation of the model quality on the test set
print("Predicting people's names on the test set\n")
# Predictin for the test set
y_pred = clf.predict(X_test_pca)

# Prediction results
print(classification_report(y_test, y_pred, target_names=target_names))

### .e Plotting the prediction
Here are a few functions so that, we can visualize the images with true label and predicted label.

In [0]:
def plot_gallery(images, titles, h, w, n_row=3, n_col=5):
    """Helper function to plot a gallery of portraits
    
       inputs:
          images (numpy array) : array of images that we want to show
          titles (string array) : 
          h (int) : height of the image
          w (int) : width of the image
          n_row (int) :
          n_col (int) :
    """
    plt.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(images[i].reshape((h, w)), cmap=plt.cm.gray)
        plt.title(titles[i], size=12)
        plt.xticks(())
        plt.yticks(())
        

def title(y_pred, y_test, target_names, i):
    """Helper function to generate the title
    inputs:
      y_pred : List of predictions 
      y_test : 
    
    outputs:
    """
    pred_name = target_names[y_pred[i]].rsplit(' ', 1)[-1]
    true_name = target_names[y_test[i]].rsplit(' ', 1)[-1]
    return 'predicted: %s\ntrue:      %s' % (pred_name, true_name)


Let's visualize a few images from the dataset to see the prediction results.

In [0]:
# Generate titles with predicted and true labels for the images
prediction_titles = [title(y_pred, y_test, target_names, i) for i in range(y_pred.shape[0])]
# Call the plot function with necessary parameters
plot_gallery(X_test, prediction_titles, h, w)

plt.show()

**Optional:** We already have seen the prediction output. One last thing we can do, is visualize the **eigenfaces**, reconstructed from the `PCA` output. Let's look at a few images, how they look like!

In [0]:
# plot the gallery of the most significative eigenfaces
eigenface_titles = ["eigenface %d" % i for i in range(eigenfaces.shape[0])]

plot_gallery(eigenfaces, eigenface_titles, h, w)

plt.show()

**Your Turn:** We have used `n_components = 150` for `PCA`. Now you need to tweak the parameter with different values and select the best one. You may consider looking at the average *precision, recall and f1-score* from the classification report..

**Your Turn (Question 2):** Which of the following values for `n_components` in PCA gives you better classification result?

_Choose from: 50, 150, 300_

**Your Turn (Question 3):** Why you get good result for stated `n_components` value in the previous question? Explain your answer.

_Replace with your answer_

## 2 Programming : $k$-means Clustering


In this exercise, we will implement basic $k$-means clustering algorithm from scratch. We will see limitations of the $k$-means algorithm and try to improve on them.

### .a Setup
At the beginning of our implementation we will first `import` necessary libraries.

In [0]:
# import necessary libraries
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import seaborn as sns; sns.set()

We will generate data points for our exercise using the `sklearn` library. For this exercise, we will be using $4$ clusters as our input data. 

In [0]:
from sklearn.datasets.samples_generator import make_blobs

# Generating 300 data points with 4 centers
X, y_true = make_blobs(n_samples=300, centers=4, cluster_std=0.60, random_state=0)

# Plot the generated data points
plt.scatter(X[:, 0], X[:, 1], s=50);

### .b $k$-means from `sklearn`

Before implementing the $k$-means algorithm from scratch, let's have a look at the `sklearn` version.

In [0]:
# Import KMeans from sklearn
from sklearn.cluster import KMeans

# Creating an instance of K-means for 4 clusters
kmeans = KMeans(n_clusters=4)
kmeans.fit(X)

# Get the prediction for the data points
y_kmeans = kmeans.predict(X)

We have the prediction for our data points in `y_kmean`. Let's visualize the predicted clusters.

In [0]:
# Plot the output of sklearn version of k-means
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='red', s=200, alpha=0.5);

### .c Implementing $k$-means from scratch

We will start implementing $k$-means by ourselves. In this version of $k$-means, we will start with random **centers** for our clusters. 

The function `randomCenters` generates initial random centers for our $k$-means algorithm.

In [0]:
import random

def randomCenters(X, n_clusters, rand_seed):
  """
    Args:
      X (m x 2) : input 2D data points
      n_clusters (int) : number of clusters
      rand_seed (int) : seed for random number generation
    
    Returns:
      centers (array of floats) : random centers
  """
  # Size of our data set
  sz = len(X)
  
  # Use the random seed to get same output for everyone
  random.seed(rand_seed)
  
  # Generate four (n_clusters) random positions
  positions = random.sample(range(1,sz), n_clusters)
  
  # Assign those points as initial centers
  centers = X[positions]
  
  return centers

  `plotCenter` is another helper function to plot the centers and the cluster assignments.

In [0]:
def plotCenter(X , centers, labels, titleNumber,  titleStr):
  """
    Args:
      centers (array of float) : array of centers
      labels (array of integer) : cluster assignment of data points
      titleNumber (int) : Random seed for current iteartion or iteration number to show in the title
      titleStr (string) : Plot title
      
  """
  plt.scatter(X[:, 0], X[:, 1], c=labels, s=50, cmap='viridis');
  plt.scatter(centers[:, 0], centers[:, 1], c='red', s=200, alpha=0.5);
  plt.suptitle("%s %d" % (titleStr,titleNumber))
  plt.show()

The function `pairwiseDistance` calculates the distance between two given data points. We will use this function to calculate the nearest cluster and assign the point to the nearest cluster.

In [0]:
def pairwiseDistance(x1, x2):
  """
    Args:
      x1 (2D point) : one data point
      x1 (2D point) : another data point
    Returns:
      dist (float) : distance between the two data points
  """
  dist = np.linalg.norm(x1-x2, axis=0)
  return dist

Great! Now we have everything we need. All we need to do now is to implement the $k$-means algorithm. Let's recall the $k$-means algorithm first.

**$k-$ _means algorithm:_**

$
\textbf{Data: } \text{a set of points, $x^{(j)} \in \mathbb{R}^n$}\\
\textbf{Output: } \text{a $m$-tuple of clusters, $(y^{(1)}, \dots, y^{(m)})$} \text{where each $y^{(i)} \in \{1,,2, \dots, k\}$}\\
\textbf{for } i=1 \text{ to } k \text{ do:} \\
\hspace{5mm}\mu_i \leftarrow \text{ some random location} \\
\textbf{repeat} \text{ until converged:}\\
\hspace{5mm} \textbf{for } j=1 \text{ to }m \text{ do:} \\
\hspace{10mm}y^{(j)} \leftarrow \underset{i}{\text{ arg min}} ||\mu_i - x^{(j)}||\\
\hspace{5mm} \textbf{for } i=1 \text{ to } k \text{ do:} \\
\hspace{10mm} S_i \leftarrow \{x^{(j)}|y^{(j)} = i \}\\
\hspace{10mm} \mu_i \leftarrow \frac{1}{|S_i|} \underset{x^{(j)}\in S_i}{\sum}x^{(j)}\\
\textbf{return }(y^{(1)}, \dots, y^{(m)})
$


We will now implement the algorithm above in Python. Don't worry, you don't have to write everything. We have already written most of the code, we just need a little help to finish the code. 

**Your Turn (Question 4):** Complete the code below to implement $k$-means clustering.

In [0]:
import random

def kMeansClustering(X, n_clusters, centers, plot_steps = 0, max_iter = 100):
    """
    Args:
      X (mx 2): Input data points
      n_clusters (int) : number of clusters
      centers (array of float) : initial centers
      plot_steps (int) : if set to 1 shows each iteration of the algorithm, deafult value 0
      max_iter (int) : maximum number of iteration, deafult value 100
    
    Returns:
      labels (m x 1) : cluster assignment of each point
      centers (array of float) : final centers of the clusters
    """
    
    # Length of our data set
    sz = len(X)
    
    # Initializing labels array, where we will store the cluster assignment
    labels = np.zeros(sz)
    
    # Loop until max iteartion and recalculate center at each step
    for iterator in range(max_iter):
        # Check if we want to show each iteration of the algorithm
        if plot_steps:
            plotCenter(X, centers, labels, iterator, "Iteration")
        
        #########################################
        # Your Turn: write code here
        #
        # Loop through each point
        # For each point : calculate distance for different clusters
        # Assign the cluster with minimum distance in labels array
        #
        # Recalculate the centers, store in new_centers from the means of the newly assigned points 
        #
        #########################################
        
        # if centers are same as previous, break the loop and return current centers and labels
        if np.all(centers == new_centers):
            break
          
        centers = new_centers
    
    return labels, centers   

Let's check if your implementation works!

In [0]:
# Assign number of clusters
n_clusters = 4

# Initial centers with seed 11, so that everybody can have same answers
centers = randomCenters(X, n_clusters, rand_seed=11)

# Calling our implemented function, with plot_steps set to 1 so that we can see each iteration
labels, new_centers = kMeansClustering(X, n_clusters, centers, plot_steps=1)
#print(new_centers)

### .d Run $k$-means with different random seed

Great work! Our implementation seems to work fine. But does it always give correct result? In this implementation, we randomly assign our initial centers. Let's try with different initialization, by having different *random seed*.  In the code snippet below, we will use $5$ different seeds and visualize the final output. Let's execute the code and see the result!

In [0]:
# Assign number of clusters
n_clusters = 4

# Five different seeds
seeds = [19, 21, 27, 29, 85]

# Run our K-means for each seed
for rseed in seeds:
    
    # initialize random centers
    centers = randomCenters(X, n_clusters, rseed)
    
    # output the K-means clustering
    labels, new_centers = kMeansClustering(X, n_clusters, centers)

    # plot the final output of different seeds
    plotCenter(X, new_centers, labels, rseed, "Cluster output with seed: ")

**Your Turn:** Execute the above code (for different random seeds value) and observe the results. Does the result looks same all the time? Can it successfully identify all the clusters?

**Your Turn (Question 5):** Does the algorithm successfully identify the clusters each time?

_Choose from: Yes, No_

**Your Turn (Question 6):** Explain why we got this type of output.

_Replace with your answer_

**Your Turn:** In the above implementation, we stop when the `new_centers` of the clusters are the same as the previous step, `centers`; i.e. if the center of the cluster doesn't change from previous step, we stop and return those as the final cluster centroid. 

Let's assume, we could not identify the clusters successfully and got the output as the figure shown below. What if we **remove** that part of code (remove the `break`) and continue to run the code till `max_iter`?

![alt text](https://www.comp.nus.edu.sg/~neamul/Images/week12.PNG)

**Your Turn (Question 7):** What result will we get, if we run the code for `max_iter` iterations?

_Choose from: The result will be better than previous; The result will get worse than previous; The result may get better if we increase the value of `max_iter`; It will remain same as previous, no matter how many times we run_

## 3 Apply $k$-means on Moon dataset

We have seen two different version of $k$-means so far. Now we will apply our algorithm to a completely new dataset; the moon dataset. Let's generate the moon dataset.

In [0]:
from sklearn.datasets import make_moons
# Generate moon dataset
Xmoon, ymoon = make_moons(200, noise=.05, random_state=0)
# Plot the data points
plt.scatter(Xmoon[:, 0], Xmoon[:, 1], s=50);

You can try both `sklearn` version of $k$-means or our implemented one and see the output.

In [0]:
from sklearn.cluster import KMeans
## To run the sklearn version 

labels = KMeans(2, random_state=0).fit_predict(Xmoon)
plt.scatter(Xmoon[:, 0], Xmoon[:, 1], c=labels,s=50, cmap='viridis');
plt.suptitle("k-means on moon data : sklearn version")
plt.show()
plt.close()
n_clusters = 2

# Our implemented version
rand_seed = 11
centers = randomCenters(Xmoon, n_clusters, rand_seed)
labels, new_centers = kMeansClustering(Xmoon, 2, centers)
plotCenter(Xmoon, new_centers, labels, 11, "k-means on moon data with seed: ")

**Your Turn (Question 8):** How can we improve the result for moon dataset?

_Replace with your answer_

---
# Week 11: Post-tutorial Work

Watch week $11$ post-videos and go through the exercise in the notebook.

_Note:_ we re-use certain variables, (e.g., $X$) in the post tutorial notebook.  Please try to run the required portion of the notebook before running certain blocks.

## 4 Programming : $k$-means++

As you can see from the pre-tutorial notebook, $k$-means does not return the correct output each time. Now we will try to modify our center intialization using the following $k$-means++ algorithm.

**$k$ _means++ algorithm:_**

$
\mu_1 \leftarrow \text{randomly chosen} \\
\textbf{for } i=2 \text{ to } k \text{ do:} \\
\hspace{6mm} \textbf{for } j = 1 \text{ to } m \text{ do:} \\
\hspace{12mm}d^{(j)} \leftarrow \text{ min}_{i'<i} ||x^{(j)} - \mu_{i'}||
\hspace{6mm} \text{ ## compute distance}\\ 
\hspace{6mm} P: p(j) = \frac{d^{(j)}}{\sum_j d^{(j)}}, j \in \{ 1, \cdots, m \}\hspace{12mm} \text{ ## make a probability distribution } \\
\hspace{6mm}c \sim P, c\in \{1, \dots, m\} \hspace{6mm}\text{##Use $P$ to pick a point as the center}\\
\hspace{6mm}\mu_k \leftarrow x^{(c)} \\
$

Run $k$-means using $\mu_{1...k}$ as the initial centers.


### .a Choosing initial centers

We will now implement this code for our initial centers. We have written some part of the code, but we left out the part that calculates the distance and the probabilities.

Finish the code block below with the necessary code.

**Your Turn (Question 1):** Complete the code below to initialize cluster centers according to $k$-means++ algorithm.

In [0]:
from scipy import stats

def initialCenters(X, n_clusters, rand_seed):
    """
    Args:
      X (N x 2) : input data points
      n_clusterts (int) : number of clusters
      rand_seed (int) : seed for random number generation
    Returns:
      centers (array of float) : initial centers based on probability proportional to dist
    """
    
    # Length of our dataset
    sz = len(X)
    
    # Fixing the random seed to have same output for everyone
    random.seed(rand_seed)
    
    # Generate first center randomly
    pos = random.sample(range(1, sz), 1)
    centers = X[pos]
    
    for i in range(n_clusters-1):
        dist = np.zeros(sz)
        # The following two lines are just for initialization. You can remove those if you don't need
        prob = np.zeros(sz)
        prob[0] = 1
        #######################################
        # Your Turn: write your code here
        #
        # For each point:
        #    calculate distance for current centers (initially 1)
        #    get the minimum dist and store them in dist
        #
        # 1. Calculate sum of all distances
        # 2. Calculate probability from dist and sum_of_all_dist
        # 3. Store the probability in `prob` as np array
        #
        #######################################
        
        # Generate random number from our custom prob distribution 
        custom = stats.rv_discrete(name='custm', values=(np.arange(sz), prob))
          
        pos = custom.rvs(size=1)
        # Generate new center using the generated random number and append it with previous centers
        centers = np.append(centers, [X[pos[0]]], axis = 0)
    
    return centers

### .b Setup
Let's generate our data points using `blobs`.

In [0]:
# import necessary libraries
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import seaborn as sns; sns.set()

from sklearn.datasets.samples_generator import make_blobs
X, y_true = make_blobs(n_samples=300, centers=4, cluster_std=0.60, random_state=0)

### .c Run the $k$-means

Now we will try our $k$-means using this center initialization. Let's see if we can get better result or not.

Make sure you run the **`kMeansClustering()`** function, that you implemented in **Section $2c$**. We are using the same implementation with modified center intialization.

In [0]:
# Assign number of clusters
n_clusters = 4


# Seed for random number generation for first center; you can change it to different value to see if it works or not
rand_seed = 19

# Substitute in new center initialization
centers = initialCenters(X, n_clusters, rand_seed)
# print(centers)
# For reference, this was the old code block from the beginning that called the original version.
# centers = randomCenters(X, n_clusters, 11)

# Using the same k means with new centers
labels, new_centers = kMeansClustering(X, n_clusters, centers)

# Plot the final output
plotCenter(X, new_centers, labels, rand_seed, "K-means++ with seed: ")

The output looks great. You can try it with any random seed, the result should be better than the basic $k$-means algorithm.

## 5 Problems with $k$-Means Algorithm

In this part we will look at a few **problems** of $k$-means and will try to improve the result using the **Gaussian Mixture Model**. In the pre-tutorial exercise, we have seen $k$-means can cluster the points pretty easily (within a few iterations). 



### .a Run $k$-Means

We will use the same dataset (from Section $4b$) to run the `sklearn` version of $k$-means. We have already did that previously, let's do it once more and discuss some problems!

In [0]:
# import sklearn library for kmeans
from sklearn.cluster import KMeans
# create an instance of kmeans with 4 clusters
kmeans = KMeans(4, random_state=0)
# gather cluster output for our data points
labels = kmeans.fit(X).predict(X)
# plot the cluster output
plt.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis');

### .b Problem with cluster assignment
Intuitively, we can see from the plot, the assignment of some points we are confident about the assignment of clusters of some points more than others. For example, we are more confident of assigning points that lie close to the center of the clusters correctly than the points lying close to other cluster boundary. From the $k$-means algorithm, we have seen that the algorithm doesn't have any measure of uncertainty or probability of cluster assignments. 

One easy way of thinking of $k$-means algorithm is that it places a circle (or hypersphere, in higher dimensions) at the center of the each cluster. The radius of those circles is defined by the most distant point in the cluster. This radius is a **hard** cut-off for a cluster; _i.e._, any point that lies inside the radius is included in the cluster; otherwise, it is not. 

We can visualize the cluster boundaries using circles. There is a slight overlap between the two clusters in the middle, which we can't see from the above figure. We will try to visualize it using the circles.

In [0]:
from scipy.spatial.distance import cdist

def plot_kmeans(kmeans, X, n_clusters=4, rseed=0, ax=None):
    """
      Args:
        kmeans : the instance of our kmeans classifier
        X (m x 2) : input data points
        n_clusters (int) : number of clusters
        rseed (int) : seed for random number
        ax : axis of the data points for plotting
    """
    # Get the cluster assignment
    labels = kmeans.fit_predict(X)

    # plot the input data
    ax = ax or plt.gca()
    ax.axis('equal')
    #ax.set_xlim(-5,5)
    ax.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis', zorder=2)

    # plot the representation of the KMeans model
    centers = kmeans.cluster_centers_
    radii = [cdist(X[labels == i], [center]).max() for i, center in enumerate(centers)]
    # Different hexcodes for different clusters
    hexCodes = ['#CCEEBB', '#CCAACC', '#CCFFDD', '#AABBCC']
    for c, r, clr in zip(centers, radii, hexCodes):
        ax.add_patch(plt.Circle(c, r, fc=clr, lw=3, alpha=0.5, zorder=1))

In [0]:
# Create an instance of k means for 4 clusters
kmeans = KMeans(n_clusters=4, random_state=0)
# Plot the cluster with boundary
plot_kmeans(kmeans, X)

Note: The points may seem a bit closer than the original data points. It is because of the scaling of our figure, we now have equal sized axes now to visualize the circle better. Previously, the range for both axes were different. The data points are actually same, only the range of the figure changed.

### .c $k$-means on elliptical clusters
From the plot above, it is visible that there is a slight overlap between the two clusters in the middle. Now, think of a point which is equidistant from two or more clusters. As we implemented our $k$-means algorithm, we first calculate distance from a cluster and then we check if it is less than previous minimum. If the distance is strictly less than other centers we assign the point to that cluster. That means that if we have a point equidistant from two clusters, it will always be assigned to the first cluster. 

Is it necessary correct way to assign a point to a cluster when there is a point that is equidistant from both the clusters? Intuitively, we would think that it is equally likely for the point to belong to both clusters. The $k$-means algorithm doesn't consider fully capture the dilemma in this scenario. 




Another observation from the above plot, $k$-means always think the clusters are **circular**, it doesn't take into consideration that the cluster can be of other shapes such as *ellipse* or *oblong*.

We can use the same data and transform it to be stretched out. Then we apply $k$-means and visualize the clusters. From the plot, you can understand the point above better.

In [0]:
# Transform the original data points to more stretched points
rng = np.random.RandomState(13)
X_stretched = np.dot(X, rng.randn(2, 2))

# Get clustering for the transformed data points
kmeans = KMeans(n_clusters=4, random_state=0)

# Plot the cluster output
plot_kmeans(kmeans, X_stretched)

In the figure above, there is a huge overlap between the clusters, which is not good. When the clusters doesn't form a circular shape, $k$-means doesn't seem to perform poorly. So we need a clustering method that is more general, not limited to only circles. Instead of assigning to the closest cluster, we may consider the distances of each point to all cluster centers. In the next section, we will try to use this ideas to improve our cluster assignments.

**Your Turn (Question 2):** Which of the following are disadvantages of $k$-means?

_Choose from: Easy to implement; Lack of flexibility in cluster shape; With huge number of variable, computing $k$ means costs more; Doesn't have probabilistic assignment_

## 6 Gaussian Mixture Models

In this section, we will learn *Gaussian Mixture Model* (GMM), which attempts to find a mixture of multi-dimensional Gaussian probability distributions that best model any input dataset. 



### .a GMM for clustering

Let's first use the `GMM` package from `sklearn` for finding clusters (in an identical manner as using $k$-means).

In [0]:
# import libraries for GMM
from sklearn import mixture

# Create an instance of GMM with 4 components
gmm = mixture.GaussianMixture(n_components=4).fit(X)

# Get cluster assignment for input data
labels = gmm.predict(X)

# Plot the cluster output
plt.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis');

### .b Probability based cluster assignment


The output looks similar to what we had previously, right? But it's actually not. Though `GMM` finds cluster like $k$-means, but `GMM` contains a probabilistic model under the hood. `GMM` uses probabilistic cluster assignment. For each point, it has probability of assigning it to different clusters.

We can look at some of these probabilities for a few points using `sklearn`.

In [0]:
# Get prediction probability of input data points
probs = gmm.predict_proba(X)
# Print probability of first 10 points
print(probs[:10].round(3))

From the above output, we can see the `probability matrix` of first $10$ points. For each point, it gives a row vector that specifies the probability of assigning the point to each of the 4 different clusters.

So till now, we have just talked about how `GMM` assigns a point to a cluster. But from the output we cannot distinguish. Let's know some more details about a *Gaussian Mixture Model*.

**Your Turn (Question 3):** What happens when a sample have $0.5$ probability for two clusters?

_Choose from: It will be considered as outlier, We will ignore this point in our clustering, It will be assigned to both cluster, We will assign it to any one of these two, We will assign it to any one random cluster among all_

### .c Gaussian Mixture Model : Expectation-Maximization



A Gaussian mixture model is very similar to $k$-means, in the sense that it uses an expectation-maximization approach which follows the following steps:

1. Choose starting guesses for the location and shape
2. Repeat until convergence:

    a) *E-step*: for each point, find weights encoding the probability of membership in each cluster;
    
    b) *M-step*: for each cluster, update its location, normalization and shape, based on all data points in the cluster, making use of the weights.
    
The output of this algorithm is that each cluster is associated not with a hard-edged sphere, but with a smooth Gaussian model. Just like in $k$-means, this algorithm may also sometime miss the global optimum, depending on the initialization of the centers.
    
    

### .d Visualize GMM Cluster boundaries
The function below will help us to visualize the locations and shapes of the `GMM` clusters by drawing ellipses based on the `GMM` output.

In [0]:
from matplotlib.patches import Ellipse

def draw_ellipse(position, covariance, ax=None, **kwargs):
    """Draw an ellipse with a given position and covariance
      Args:
        position : given position of the cluster
        covariance: covariance matrix of GMM
        ax : axis of the data points for plotting
        kwargs : weights of the cluster points
        
    """
    ax = ax or plt.gca()
    
    # Convert covariance to principal axes
    if covariance.shape == (2, 2):
        U, s, Vt = np.linalg.svd(covariance)
        angle = np.degrees(np.arctan2(U[1, 0], U[0, 0]))
        width, height = 2 * np.sqrt(s)
    else:
        angle = 0
        width, height = 2 * np.sqrt(covariance)
    
    # Draw the ellipse
    for nsig in range(1, 4):
        ax.add_patch(Ellipse(position, nsig * width, nsig * height, angle, **kwargs))
        
def plot_gmm(gmm, X, label=True, ax=None):
    """
      Visualize the output of a fitted GMM
      Args:
        gmm : instance of GMM classifier
        X (N x 2): input data points
    """
    ax = ax or plt.gca()
    labels = gmm.fit(X).predict(X)
    if label:
        ax.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis', zorder=2)
    else:
        ax.scatter(X[:, 0], X[:, 1], s=40, zorder=2)
    ax.axis('equal')
    
    w_factor = 0.2 / gmm.weights_.max()
    for pos, covar, w in zip(gmm.means_, gmm.covariances_, gmm.weights_):
        draw_ellipse(pos, covar, alpha=w * w_factor)

Let's plot the `GMM` cluster output using the above helper functions.

In [0]:
# Creating an instance of GMM with 4 components
gmm = mixture.GaussianMixture(n_components=4, covariance_type='full', random_state=42)
# Plot output of GMM
plot_gmm(gmm, X)

### .e GMM on Elliptical Clusters
We have seen that $k$-means cannot fit the same points, when we transform them and make them stretched. For stretched points, the clusters overlap with each other a lot. Now we will try to fit those same (stretched) points using `GMM`. Let's see if `GMM` can fit these points better than $k$-means or not.

In [0]:
# Creating GMM instance with covariance type full; you can also use spherical or diag
gmm = mixture.GaussianMixture(n_components=4, covariance_type='full', random_state=42)
# Plot the output of GMM
plot_gmm(gmm, X_stretched)

Great! The output looks way better than $k$-means. Because `GMM` doesn't try to fit the data in circles, as we can see the output it fits the data using **ellipse**. That is also why `GMM` performs better than $k$-means.

Now, we have learned two different ways of cluster assignment.

**Your Turn (Question 4):** What is the main difference between *hard clustering* and *soft clustering*?

_Replace with your answer_

**Your Turn (Question 5):** Which one *(Hard clustering and soft clustering)* is better and why? 

_Replace with your answer_

## 7 Application of GMM

In the previous section, we have used `GMM` for clustering, because it is a clustering algorithm. But `GMM` is also used for *density estimation*; i.e. estimating the underlying distribution of the sample points, which we can use to generate new points.



### .a Setup
We will see this application of `GMM` with the two moons dataset. Let's generate the dataset first.

In [0]:
from sklearn.datasets import make_moons
# Generate moon data points
Xmoon, ymoon = make_moons(200, noise=.05, random_state=0)
# Plot the data points
plt.scatter(Xmoon[:, 0], Xmoon[:, 1])

### .b Learning the density function
If we apply `GMM` with only **two components**, the output looks poor (In the previous example, we have used 4 components, because we had four clusters, whereas we now have only two).

In [0]:
# Create instance of GMM with 2 components; same as before
gmm2 = mixture.GaussianMixture(n_components=2, covariance_type='full', random_state=42)

# Plot output of GMM
plot_gmm(gmm2, Xmoon)

So the output doesn't seem to fit the data so well. Its because we have used only two components to estimate the density of the data. Now we will try to increase the number of components and see if our model can fit the underlying distribution or not. We will be using $16$ components for the moon dataset.

In [0]:
# Create instance of GMM with 16 components
gmm16 = mixture.GaussianMixture(n_components= 16, covariance_type='full', random_state=0)
# Plot the output of GMM
plot_gmm(gmm16, Xmoon, label=False)

From, the above output, it seems like our model has learned most of the data points. But from this figure we cannot judge how well it can learn the density function. 



### .c Generating new points
We will generate 1000 points using the learned `GMM` model, and compare the generated points with the original dataset.

In [0]:
# Generate new data points from the learned GMM
Xnew=gmm16.sample(n_samples=1000)
# Plot generated data points
plt.scatter(Xnew[0][:, 0], Xnew[0][:, 1]);

It looks quite good. Our model seems to have learned the distribution of the original dataset. This is a very useful application of `GMM`.  We can use this model to generate data points of an unknown distribution, which may help us in our learning algorithms.

**Your Turn (Question 6):** In which of the following situation will generating more data points will be helpful?

_Choose from: Detecting expression from comments on social media, Detecting useful keyword from news blogs, Cancer detection where we have lots of negative data points only a few positive points_

**N.B.** The post exercise is adapted from Python Data Science NoteBook on [Gaussian Mixture Model](https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/05.12-Gaussian-Mixtures.ipynb).

---
# Credits
Authored by Mohammad Neamul Kabir and  [Min-Yen Kan](http://www.comp.nus.edu.sg/~kanmy) (2019), affiliated with [WING](http://wing.comp.nus.edu.sg), [NUS School of Computing](http://www.comp.nus.edu.sg) and [ALSET](http://www.nus.edu.sg/alset).
Licensed as: [Creative Commons Attribution 4.0 International](https://creativecommons.org/licenses/by/4.0/ ) (CC BY 4.0).
Please retain and add to this credits cell if using this material as a whole or in part.
Other Credits (inclusive of photos): 